# Hierarchy Scores

> This module contains a Python implementation of the Krackhardt hierarchy score.

In [ ]:
#| default_exp hierarchy

In [ ]:
#| hide
from nbdev.showdoc import *
import os, pandas as pd, numpy as np, networkx as nx
from itertools import product

import rpy2, rpy2.situation
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr

from pykrack.utils import graph_properties

# Introduction

Biological signaling can be modeled as a directed network, where nodes represent genes/proteins and edges represent signaling interactions. 

The hierarchy of such a network can be quantified using various metrics, including the Krackhardt hierarchy score. This score measures the degree to which the network exhibits a perfect hierarchy, with higher scores indicating greater hierarchy. 


In R the `sna` package presents methods to compute graph hierarchy including Krackhardt's score (defined as $A/MaxV$ where $A$ is the number of unordered pairs of points in Dr that are asymmetrically linked), and there are other hierarchy scores implemented in Python such as *Flow Hierarchy Score* [Luo and Magee 2011](https://doi.org/10.1002/cplx.20368).
However, despite its utility, there is currently no native implementation of the Krackhardt hierarchy score in Python. 

## Krackhardt hierarchy score

As defined in *Krackhardt, David. (1994). Graph Theoretical Dimensions of Informal Organization. Computational Organization Theory. 89*. 

    The graph hierarchy condition states that in a digraph D, for each pair of points where one (Pi) can reach another (Pj), the second (Pj) can't reach the first (Pi). 
    For example, in a formal organization chart a high lvl employee can reach through the chain of command her subordinate's subordinate. If the formal organization is working "properly", this lower lvl employee can't simultaneously reach the high lvl employee.
    To measure the degree of hierarchy of digraph D, a new digraph Dr must be created. Dr is defined as the reachability digraph of D. Each point in D exists in Dr; moreover, the line (Pi,Pj) exists in Dr if and only if Pi can reach Pj in D. If D is graph hierarchic, then Dr will have no symmetric lines in it (i.e. if the line (Pi,Pj) exists in Dr then the line (Pj,Pi) does not).

The degree of hierarchy then is defined as:

$$
    Graph Hierarchy = 1 - [V/MaxV]
$$
Where
    $V$ is the number of unordered pairs of points in Dr that are symmetrically linked
    , and $MaxV$ the number of unordered pairs of points in Dr where Pi is linked to Pj or viceversa.

<!-- **Definitions**:

- Hierarchy degree: This a network-wide metric that determines how "hierarchical" a graph is.
From https://doi.org/10.1186/s13059-015-0624-2:
    The degree of hierarchy for a given for a given network is not well-defined concept. Ispolatov et al. [27] introduced the idea of dominant direction by minimizing the number of feedback links. While it is a proxy of hierarchical structure to a certain extent, the method does not provide a rigorous statistical confidence. Here, we define a metric to quantify the degree of hierarchy for a given hierarchical network, and then propose a new method called hierarchical score maximization (HSM) to infer the hierarchy of a directed network

- Hierarchy metric (at node level): From sna package in R.
    Hierarchy measures quantify the extent of asymmetry in a structure; the greater the extent of asym-
    metry, the more hierarchical the structure is said to be. (This should not be confused with how
    centralized the structure is, i.e., the extent to which centralities of vertex positions are highly con-
    centrated.)
They discuss and offer 2 different approaches:

    - reciprocity: This setting returns one minus the dyadic reciprocity for each input graph (see
    grecip)
    - krackhardt: This setting returns the Krackhardt hierarchy score for each input graph. The
    Krackhardt hierarchy is defined as the fraction of non-null dyads in the reachability graph
    which are asymmetric. Thus, when no directed paths are reciprocated (e.g., in an in/outtree),
    Krackhardt hierarchy is equal to 1; when all such paths are reciprocated, by contrast (e.g., in
    a cycle or clique), the measure falls to 0.
        - Hierarchy is one of four measures (connectedness, efficiency, hierarchy, and lubness)
        suggested by Krackhardt for summarizing hierarchical structures. Each corresponds to one of
        four axioms which are necessary and sufficient for the structure in question to be an outtree;
        thus, the measures will be equal to 1 for a given graph iff that graph is an outtree. Deviations
        from unity can be interpreted in terms of failure to satisfy one or more of the outtree conditions,
        information which may be useful in classifying its structural properties.
        Note that hierarchy is inherently density-constrained: as densities climb above 0.5, the proportion
        of mutual dyads must (by the pigeonhole principle) increase rapidly, thereby reducing possibili-
        ties for asymmetry. Thus, the interpretation of hierarchy scores should take density into account,
        particularly if density is artifactual (e.g., due to a particular dichotomization procedure)
    
- Flow Hierarchy Score: Described in XXXX (paper). Implementation from the networkX package. -->

In [ ]:
for i in rpy2.situation.iter_info(): # Print Rpy2 info
    print(i)

pandas2ri.activate()

rpy2 version:
3.5.11
Python version:
3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:39:40) [Clang 15.0.7 ]
Looking for R's HOME:
    Environment variable R_HOME: /Users/ferran/miniconda3/envs/pykrack/lib/R
    Calling `R RHOME`: /Users/ferran/miniconda3/envs/pykrack/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/local/lib/R/library/stats/libs/:/usr/local/lib/R/library/stats/libs/
R version:
    In the PATH: R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/Users/ferran/miniconda3/envs/pykrack/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'dl', 'm', 'iconv', 'icuuc', 'icui18n']
  library_dirs:
  ['/Users/ferran/miniconda3/envs/pykrack/lib', '/Users/ferran/miniconda3/envs/pykrack/lib/R/lib', '/Users/ferran/miniconda3/envs/pykrack/lib']
  extra_compile_args:
  ['-std=c99']
  extra_link_arg

In [ ]:
#| export
def compute_hierarchy(G, metric="pykrack"):
    """
    Compute one of the possible hierarchy scores
    
    Parameters
    ----------
    G
        Directed NetworkX graph
    metric : str
        Type of hierarchy metric to compute. Accepted types are:
        'pykrack' for this module's implementation of the Krackhardt score.
        'rsnakrack' for the sna implementation in R.
        'hierarchy_flow' for the Luo and Magee 2011 as implemented in the NetworkX package.

    Returns
    -------
    score : float
        One of the possible hierarchy scores
    """
    
    import networkx as nx # Be mindful of scoping!
    from itertools import product

    #Ensure Graph is DirectedGraph
    if not G.is_directed():
        raise Exception
    #Ensure Graph is of DiGraph() format
    G = nx.DiGraph(G)

    if metric == "pykrack": #Python implementation
        #Compute transitive closure of graph to get the reachability graph
            #[contains an edge (i,j) if there is a path from i to j in the original graph]
        acyclic = 0
        try:
            nx.find_cycle(G)
        except:
            print("Acyclic graph")
            acyclic = 1
        if acyclic == 1:
            Gr = nx.transitive_closure_dag(G)
        else:
            Gr = nx.transitive_closure(G, reflexive=None)
        symmetric_dyads = 0
        non_null_dyads = 0
        n = len(Gr.nodes())
        #Count the number of non-null symmetric dyads
        for pair in product(Gr.nodes(), Gr.nodes()):
            if Gr.has_edge(pair[0],pair[1]) or Gr.has_edge(pair[1],pair[0]): #Non-null dyad
                non_null_dyads+=1
                if  Gr.has_edge(pair[0],pair[1]) == Gr.has_edge(pair[1],pair[0]): #Symmetric!
                    symmetric_dyads+=1
        #Raise exception if graph has no edges!
        if non_null_dyads == 0:
            raise Exception
        score = 1 - (symmetric_dyads / non_null_dyads)
    
    elif metric == "rsnakrack": #R implementation from the sna package
        #Be wary of scoping again!
        from rpy2.robjects.packages import importr

        try:
            base = importr("base")
            sna = importr("sna")
            score = sna.hierarchy(nx.to_numpy_array(G), measure="krackhardt")[0]
        except:
            print("R package sna was not found. Please install manually!")
            print("Computing hierarchy flow instead")
            snafail_flag = 1
            score = nx.flow_hierarchy(G)
    
    elif metric == "hierarchy_flow": #Networkx's hierarchy flow implementation
        score = nx.flow_hierarchy(G)

    # elif metric == "all": #This will eventually return a dict with all metrics

    else: # metric argument broken
        raise Exception



    return score


In [ ]:
show_doc(compute_hierarchy)

---

[source](https://github.com/FerranC96/pykrack/blob/main/pykrack/hierarchy.py#L7){target="_blank" style="float:right; font-size:smaller"}

### compute_hierarchy

>      compute_hierarchy (G, metric='pykrack')

Compute one of the possible hierarchy scores

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| G |  |  | Directed NetworkX graph |
| metric | str | pykrack | Type of hierarchy metric to compute. Accepted types are:<br>'pykrack' for this module's implementation of the Krackhardt score.<br>'rsnakrack' for the sna implementation in R.<br>'hierarchy_flow' for the Luo and Magee 2011 as implemented in the NetworkX package. |
| **Returns** | **float** |  | **One of the possible hierarchy scores** |

In [ ]:
G1 = nx.DiGraph([(1, 2), (2, 3), (1, 5), (2, 4), (4, 6), (5, 6), (3, 1)])

In [ ]:
G2 = nx.DiGraph([(1, 2), (2, 3), (1, 5), (2, 4), (4, 6), (5, 6)])

In [ ]:
G3 = nx.DiGraph([(1, 2), (4, 6)])

In [ ]:
#Graph from OmniPath
#using the following link: https://omnipathdb.org/interactions?genesymbols=yes&fields=type&license=academic&directed=0
#Gets us all interactions, no matter type nor undirected/vs directed
omnidata = pd.read_csv("https://omnipathdb.org/interactions?genesymbols=yes&fields=type&license=academic&directed=1", sep="\t")

G4 = nx.from_pandas_edgelist(omnidata, source="source_genesymbol", target="target_genesymbol",edge_attr="type", create_using=nx.DiGraph)

In [ ]:
G = G1

print(graph_properties(G))

Edges:7, Nodes:6, Avg Degree:2.3333333333333335, Density:0.23333333333333334


Using `product` from `itertools` is around 4 times faster when iterating through the nodes of the Omnipath graph.

In [ ]:
# %%time
# countlist = []
# for i in G.nodes():
#     for j in G.nodes():
#         pair = [i,j]
#         countlist.append(pair)

# print(len(countlist))

In [ ]:
# %%time
# countlist = []
# for pair in product(G.nodes(), G.nodes()):
#     countlist.append(pair)

# print(len(countlist))

In [ ]:
%%time

compute_hierarchy(G, metric="pykrack")

CPU times: user 232 µs, sys: 22 µs, total: 254 µs
Wall time: 257 µs


0.7857142857142857

In [ ]:
%%time

compute_hierarchy(G, metric="rsnakrack")

CPU times: user 972 ms, sys: 103 ms, total: 1.08 s
Wall time: 1.16 s


0.7857142857142857

In [ ]:
%%time

compute_hierarchy(G, metric="hierarchy_flow")

CPU times: user 323 µs, sys: 19 µs, total: 342 µs
Wall time: 349 µs


0.5714285714285714

In [ ]:
#| hide
try:
    assert round(compute_hierarchy(G, metric="pykrack"),2) == round(compute_hierarchy(G, metric="rsnakrack"),2)
except AssertionError:
    print("WARNING: This message should only appear when the R package sna is not installed")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()